## Benchmarking

In [1]:
import time
import autohf as hf
import numpy as np
import autograd.numpy as anp
import autograd

In [2]:
i = np.array([1, 1])
j = np.array([1, 1])
t = np.array([1, 1])

Ra = np.array([1.0, 1.0])
Rb = np.array([0.0, 0.0])
alpha, beta = np.array([1.0, 1.0]), np.array([0.5, 0.5])

np.vectorize(hf.expansion_coeff)(i, j, t, Ra, Rb, alpha, beta)

array([0.07961459, 0.07961459])

In [2]:
L1, L2, L3, L4 = (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1)
alpha, beta, gamma, delta = 1.0, 0.5, 0.2, 0.1
Ra, Rb, Rc, Rd = np.array([1.0, 0.0, 0.0]), np.array([1.0, 1.0, 0.0]), np.array([0.0, 1.0, 0.0]), np.array([0.0, 0.0, 0.0])

In [5]:
%timeit autograd.grad(hf.electron_repulsion)(alpha, L1, Ra, beta, L2, Rb, gamma, L3, Rc, delta, L4, Rd)

17.7 s ± 1.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit autograd.grad(hf.electron_repulsion)(alpha, L1, Ra, beta, L2, Rb, gamma, L3, Rc, delta, L4, Rd)

14.5 s ± 2.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%timeit autograd.grad(hf.electron_repulsion)(alpha, L1, Ra, beta, L2, Rb, gamma, L3, Rc, delta, L4, Rd)

20.7 s ± 6.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%timeit autograd.grad(hf.electron_repulsion)(alpha, L1, Ra, beta, L2, Rb, gamma, L3, Rc, delta, L4, Rd)

19.9 s ± 4.67 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def electron_repulsion(alpha, L1, Ra, beta, L2, Rb, gamma, L3, Rc, delta, L4, Rd):
    """Electron repulsion between Gaussians"""
    l1, m1, n1 = L1
    l2, m2, n2 = L2
    l3, m3, n3 = L3
    l4, m4, n4 = L4

    p = alpha + beta
    q = gamma + delta
    quotient = (p * q)/(p + q)

    P = gaussian_prod(alpha, Ra, beta, Rb) # A and B composite center
    Q = gaussian_prod(gamma, Rc, delta, Rd) # C and D composite center

    val = 0.0
    v_exp = anp.vectorize(expansion_coeff)
    for t in range(l1+l2+1):
        for u in range(m1+m2+1):
            for v in range(n1+n2+1):
                for tau in range(l3+l4+1):
                    for nu in range(m3+m4+1):
                        for phi in range(n3+n4+1):
                            val = val + v_exp(anp.array(L1), anp.array(L2), anp.array([t, u, v]), Ra, Rb, alpha, beta).prod() * \
                                   v_exp(anp.array(L1), anp.array(L4), anp.array([tau, nu, phi]), Rc, Rd, gamma, delta).prod() * \
                                   ((-1) ** tau + nu + phi) * \
                                   R(t + tau, u + nu, v + phi, 0, quotient, P - Q)

    val = val * 2 * (anp.pi ** 2.5) / (p * q * anp.sqrt(p+q))
    return val

In [ ]:
def expansion_coeff(i, j, t, Ra, Rb, alpha, beta):


    Qx = Ra - Rb
    p = alpha + beta
    q = (alpha * beta) / p

    if (t < 0) or (t > (i + j)):
        v = 0.0
    elif i == j == t == 0:
        v = anp.exp(-1 * q * (Qx ** 2))
    elif j == 0:
        v = (1 / (2 * p)) * expansion_coeff(i - 1, j, t - 1, Ra, Rb, alpha, beta) - \
            (q * Qx / alpha) * expansion_coeff(i - 1, j, t, Ra, Rb, alpha, beta) + \
            (t + 1) * expansion_coeff(i - 1, j, t + 1, Ra, Rb, alpha, beta)
    else:
        v = (1 / (2 * p)) * expansion_coeff(i, j - 1, t - 1, Ra, Rb, alpha, beta) + \
            (q * Qx / beta) * expansion_coeff(i, j - 1, t, Ra, Rb, alpha, beta) + \
            (t + 1) * expansion_coeff(i, j - 1, t + 1, Ra, Rb, alpha, beta)
    return v